# Обучение Wake Word модели в Google Colab

Этот ноутбук поможет вам обучить собственную модель пробуждения (wake word) используя openwakeword.

## 1. Настройка окружения

Установим необходимые библиотеки:

In [ ]:
# Проверяем доступность GPU
!nvidia-smi

# Клонируем репозиторий
!git clone https://github.com/YOUR_USERNAME/train-wake-word-model.git
%cd train-wake-word-model

# Устанавливаем зависимости
!pip install -q -r requirements.txt

## 2. Скачивание данных

Скачаем фоновый шум и импульсные характеристики помещений:

In [ ]:
# Скачиваем базовые данные (MIT RIRs + AudioSet)
!python download_data.py --data-dir ./data --audioset-clips 3000

# Опционально: скачать FMA для музыкального фона (~7 ГБ)
# !python download_data.py --include-fma --fma-hours 1

## 3. Настройка конфигурации

Создадим или отредактируем файл конфигурации:

In [ ]:
import yaml

# Параметры для вашей модели
config = {
    'model_name': 'my_wake_word',  # Измените на своё слово
    'target_phrase': ['hey assistant'],  # Ваше пробуждающее слово/фраза
    'custom_negative_phrases': [],
    'n_samples': 3000,  # Уменьшено для Colab
    'n_samples_val': 1000,
    'tts_batch_size': 32,
    'augmentation_batch_size': 16,
    'piper_sample_generator_path': './piper-sample-generator',
    'piper_model_path': './piper-sample-generator/models/en_US-libritts_r-medium.pt',
    'output_dir': './my_custom_model',
    'rir_paths': ['./data/mit_rirs'],
    'background_paths': ['./data/audioset_16k'],
    'background_paths_duplication_rate': [1],
    'false_positive_validation_data_path': './data/validation_set_features.npy',
    'augmentation_rounds': 1,
    'feature_data_files': {
        'ACAV100M_sample': './data/openwakeword_features_ACAV100M_2000_hrs_16bit.npy'
    },
    'batch_n_per_class': {
        'ACAV100M_sample': 1024,
        'adversarial_negative': 50,
        'positive': 50
    },
    'model_type': 'dnn',
    'layer_size': 32,
    'steps': 5000,  # Уменьшено для Colab
    'max_negative_weight': 1500,
    'target_false_positives_per_hour': 0.2,
    'target_accuracy': 0.8,
    'target_recall': 0.6
}

# Сохраняем конфигурацию
with open('./config/colab_config.yaml', 'w') as f:
    yaml.dump(config, f)

print("Конфигурация создана!")

## 4. Генерация синтетических образцов

Создадим позитивные и негативные образцы:

In [ ]:
!python train.py --training_config ./config/colab_config.yaml --generate_clips

## 5. Аугментация данных

Применим различные аугментации к образцам:

In [ ]:
!python train.py --training_config ./config/colab_config.yaml --augment_clips

## 6. Обучение модели

Запустим процесс обучения:

In [ ]:
!python train.py --training_config ./config/colab_config.yaml --train_model

## 7. Скачивание обученной модели

Скачайте модель на локальный компьютер:

In [ ]:
from google.colab import files
import os

# Путь к вашей модели (измените если нужно)
model_name = config['model_name']
model_path = f"./my_custom_model/{model_name}.onnx"

if os.path.exists(model_path):
    files.download(model_path)
    print(f"Модель {model_name}.onnx скачана!")
else:
    print(f"Модель не найдена по пути: {model_path}")
    print("Проверьте содержимое директории:")
    !ls -la ./my_custom_model/

## 8. Тестирование модели (опционально)

Протестируйте модель на аудио файле:

In [ ]:
import openwakeword
from openwakeword.model import Model
import numpy as np

# Загружаем модель
owwModel = Model(wakeword_models=[model_path])

# Тестируем на случайном позитивном примере
test_audio_path = f"./my_custom_model/{model_name}/positive_test/"
!ls {test_audio_path} | head -1

print("Модель загружена и готова к использованию!")

## 9. Очистка (опционально)

Освободите место, удалив промежуточные файлы:

In [ ]:
# Удалить данные (освободить ~5-10 ГБ)
# !rm -rf ./data/audioset_16k
# !rm -rf ./data/fma

# Удалить промежуточные аудио файлы
# !rm -rf ./my_custom_model/{model_name}/positive_train
# !rm -rf ./my_custom_model/{model_name}/negative_train

print("Для удаления файлов раскомментируйте команды выше")